In [1]:
import pandas as pd
import numpy as np

In [2]:
userFeature = pd.read_csv('./userFeature.csv')

In [ ]:
ad_static_feature = pd.read_table("./ad_static_feature.out",header=None)
adStaticFeature_data = []
with open('./ad_static_feature.out','r') as f:
    attributeName = ['aid','createTime','auid','pid','pType','aIndustryId','size']
    cnt = 0
    for i, line in enumerate(f):
        i = i + 1
        line = line.strip().split('\t')
        adStaticFeature_dict = {}
        
        if (line[1]=='0') or (len(line)<7):#creat time is 0 and size is null is dirty data
            continue
        if ',' in line[5]:# mutilty aIndustryId is dirty data 
            continue
            
        j = 0
        for each in line:
            each_list = each.split(',')
            adStaticFeature_dict[attributeName[j]] = ' '.join(each_list[0:])
#             adStaticFeature_dict[attributeName[j]] = each
            j += 1
        adStaticFeature_data.append(adStaticFeature_dict)
        # print(adStaticFeature_data)
        if i % 100000 == 0:
            print(i)
        if i % 1000000 == 0:
            ad_static_feature = pd.DataFrame(adStaticFeature_data)
            ad_static_feature.to_csv('./adStaticFeature_' + str(cnt) + '.csv', index=False)
            cnt += 1
            del adStaticFeature_data,ad_static_feature
            adStaticFeature_data = []
    ad_static_feature = pd.DataFrame(adStaticFeature_data)
    ad_static_feature.to_csv('./adStaticFeature_' + str(cnt) + '.csv', index=False)
    del adStaticFeature_data,ad_static_feature
    ad_static_feature = pd.concat([pd.read_csv('./adStaticFeature_' + str(k) + '.csv', low_memory=False) for k in range(cnt+1)]).reset_index(drop=True)
    ad_static_feature.to_csv('./adStaticFeature.csv', index=False)

In [ ]:
df_test=pd.read_csv('./adStaticFeature.csv',header=0) 

In [163]:
df_test.aid = df_test.aid.astype(np.int64)#modify all the data format
len(df_test.aid.unique())
ad_operation = pd.read_table("./ad_operation.dat",header=None)
### time cost about 10 mins
not_contain=[]
def set_contain(big, small):
    count = 0
    for t in small:
        if t not in big:
            count += 1
#             print(t, "is not contained.")
            not_contain.append(t)
#     print("{} are not contained.".format(count))
set_contain(df_test.aid.unique(), ad_operation[0].unique())
# print(not_contain)
# print(len(not_contain))
adOperation_data = []
df_test=pd.read_csv('./adStaticFeature.csv',header=0) 
# df_test.aid = df_test.aid.astype(np.str)
len(df_test.aid.unique())
with open('./ad_operation.dat','r') as f:
    attributeName = ['aid','updateTime','operationType','changeType','newValue']
    cnt = 0
    for i, line in enumerate(f):
        i = i + 1
        line = line.strip().split('\t')
        adOperation_dict = {}
        j = 0
        print(i)
        if int(line[0]) in not_contain:#the aid if is not appear in ad static ad feature is dirty data
            continue
        if line[1] == '20190230000000':#the 2.30 is not correct
            continue
            
        for each in line:
            adOperation_dict[attributeName[j]] = each
            j += 1
        adOperation_data.append(adOperation_dict)
        # print(adOperation_data)
        if i % 100000 == 0:
            print(i)
        if i % 1000000 == 0:
            ad_operation = pd.DataFrame(adOperation_data)
            ad_operation.to_csv('./adOperation_' + str(cnt) + '.csv', index=False)
            cnt += 1
            del adOperation_data,ad_operation
            adOperation_data = []
    ad_operation = pd.DataFrame(adOperation_data)
    ad_operation=ad_operation[(ad_operation[1]%1000000)==0]
    ad_operation.to_csv('./adOperation_' + str(cnt) + '.csv', index=False)
    del adOperation_data,ad_operation
    ad_operation = pd.concat([pd.read_csv('./adOperation_' + str(k) + '.csv', low_memory=False) for k in range(cnt+1)]).reset_index(drop=True)
    ad_operation.to_csv('./adOperation.csv', index=False)
# df_test.aid = df_test.aid.astype(np.int64)
df_ad_operation=pd.read_csv('./adOperation.csv',header=0) 
df_ad_operation.info()


497669

In [ ]:
ad_exposure = pd.read_table("./totalExposureLog.out",header=None)
ad_exposure.head()
# ad_exposure.info()
ad_exposure.info()
#time cost 40+mins
adExposure_data = []
with open('./totalExposureLog.out','r') as f:
    attributeName = ['aRequestId','requestTime','adLocationId','uid','aid','size','adPrice','pctr','equality_ecpm','totalEcpm']
    cnt = 0
    for i, line in enumerate(f):
        i = i + 1
        line = line.strip().split('\t')
        adExposure_dict = {}
        j = 0
        for each in line:
            each_list = each.split(',')
            adExposure_dict[attributeName[j]] = ' '.join(each_list[0:])
            j += 1
        adExposure_data.append(adExposure_dict)
        if i % 100000 == 0:
            print(i)
        if i % 1000000 == 0:
            ad_exposure = pd.DataFrame(adExposure_data)
            ad_exposure.to_csv('./adExposure_' + str(cnt) + '.csv', index=False)
            cnt += 1
            del adExposure_data,ad_exposure
            adExposure_data = []
    ad_exposure = pd.DataFrame(adExposure_data)
    ad_exposure.to_csv('./adExposure_' + str(cnt) + '.csv', index=False)
    del adExposure_data,ad_exposure
    ad_exposure = pd.concat([pd.read_csv('./adExposure_' + str(k) + '.csv', low_memory=False) for k in range(cnt+1)]).reset_index(drop=True)
    ad_exposure.to_csv('./adExposure.csv', index=False)


In [ ]:
adExposure = pd.read_csv('./adExposure.csv')

In [ ]:
adExposure = adExposure.drop_duplicates(['aRequestId','adLocationId','aid'])

In [ ]:
adStaticFeature_aid = pd.read_csv('./adStaticFeature.csv',header=0)
adStaticFeature_aid = adStaticFeature_aid[['aid']]
adStaticFeature_aid = adStaticFeature_aid.drop_duplicates('aid',keep='first', inplace=False)
adOpration_aid = pd.read_csv('./adOperation.csv',header=0)
adOpration_aid = adOpration_aid[['aid']]
adOpration_aid = adOpration_aid.drop_duplicates('aid',keep='first', inplace=False)

In [ ]:
adExposure = adExposure[adExposure['aid'].isin(adOpration_aid['aid'].values)]
adExposure = adExposure[adExposure['aid'].isin(adStaticFeature_aid['aid'].values)]
adExposure.to_csv('./adExposureNew.csv', index=False)
adExposure = pd.read_csv('./adExposureNew.csv')

In [ ]:
userFeature_data = []
with open('./user_data','r') as f:
    attributeName = ['uid','age','gender','area','status','education','consuptionAbility','device','work','connectionType','behavior']
    cnt = 0
    for i, line in enumerate(f):
        i = i + 1
        line = line.strip().split('\t')
        userFeature_dict = {}
        j = 0
        for each in line:
            each_list = each.split(',')
            userFeature_dict[attributeName[j]] = ' '.join(each_list[0:])
            j += 1
        userFeature_data.append(userFeature_dict)
        if i % 100000 == 0:
            print(i)
        if i % 1000000 == 0:
            user_feature = pd.DataFrame(userFeature_data)
            user_feature.to_csv('/Users/hanmin/Desktop/CSV/userFeature_' + str(cnt) + '.csv', index=False)
            cnt += 1
            del userFeature_data,user_feature
            userFeature_data = []
    user_feature = pd.DataFrame(userFeature_data)
    user_feature.to_csv('./userFeature_' + str(cnt) + '.csv', index=False)
    del userFeature_data,user_feature
    user_feature = pd.concat([pd.read_csv('./userFeature_' + str(k) + '.csv', low_memory=False) for k in range(cnt+1)]).reset_index(drop=True)
    user_feature.to_csv('./userFeature.csv', index=False)

In [ ]:
testSample_data = []
with open('./test_sample.dat','r') as f:
    attributeName = ['tid','aid','createTime','size','aIndustryId','pType','pid','auid','exposureTime','crowd','price']
    cnt = 0
    for i, line in enumerate(f):
        i = i + 1
        line = line.strip().split('\t')
        testSample_dict = {}
        j = 0
        for each in line:
            testSample_dict[attributeName[j]] = each
            j += 1
        testSample_data.append(testSample_dict)
        if i % 100000 == 0:
            print(i)
        if i % 1000000 == 0:
            test_sample = pd.DataFrame(testSample_data)
            test_sample.to_csv('./testSample_' + str(cnt) + '.csv', index=False)
            cnt += 1
            testSample_data = []
    test_sample = pd.DataFrame(testSample_data)
    test_sample.to_csv('./testSample_' + str(cnt) + '.csv', index=False)
    del testSample_data,test_sample
    test_sample = pd.concat([pd.read_csv('./testSample_' + str(k) + '.csv', low_memory=False) for k in range(cnt+1)]).reset_index(drop=True)
    test_sample.to_csv('./testSample.csv', index=False)